In [17]:
import re
import pandas as pd
import numpy as np

In [29]:
df = pd.read_csv("2023-06-25_data.csv")

c:\Users\patri\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3266: DtypeWarning: Columns (71,90,91) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [30]:
# Define the list of columns to drop
drop_cols = ['Posudek znalce', 'Velikost podílu', 'Minimální kupní cena', 'Plocha bazénu', 'Termín 2. prohlídky', 'Dražební vyhláška', 'Termín 1. prohlídky', 'Plocha zahrady', 'Datum konání dražby', 'Datum prohlídky do', 'Vyvolávací cena', 'Znalecký posudek', 'Minimální příhoz', 'Dražební jistota', 'Druh dražby', 'Místo konání dražby', 'Datum prohlídky', 'Anuita', 'Stav', 'Počet bytů', 'Výška stropu', 'Cena', 'Datum ukončení výstavby', 'Datum zahájení prodeje', 'Zlevněno', 'Původní cena', 'Plocha zastavěná', 'Plocha podlahová', 'Převod do OV', 'Ukazatel energetické náročnosti budovy', 'Průkaz energetické náročnosti budovy', 'Doprava', 'Komunikace' , 'Voda', 'Topení', 'Odpad', 'Telekomunikace', 'Elektřina', 'Datum nastěhování', 'Rok kolaudace', 'Rok rekonstrukce', 'ID zakázky', 'Náklady na bydlení', 'ID', 'Aktualizace', 'Poznámka k ceně']

# Define the list of columns to one-hot encode
one_hot_encode_cols = ['Typ bytu', 'Umístění objektu', 'Vybavení', 'Vlastnictví', 'Stav objektu', 'Stavba', 'Energetická náročnost budovy']

# Define the list of binary columns
binary_cols = ['Bazén', 'Půdní vestavba', 'Plyn', 'Bezbariérový', 'Terasa', 'Garáž', 'Výtah', 'Sklep', 'Parkování', 'Balkón']

# Define the list of distance columns
distance_cols = ['Večerka', 'Cukrárna', 'Divadlo', 'Veterinář',  'Hospoda', 'Kino', 'Hřiště', 'Přírodní zajímavost', 'Kulturní památka', 'Sportoviště', 'Bankomat', 'Školka', 'Pošta', 'Vlak', 'Bus MHD', 'Škola', 'Restaurace', 'Lékař', 'Tram', 'Obchod', 'Lékárna', 'Metro']


In [31]:
# Drop the unnecessary columns
df = df.drop(columns=drop_cols)

# Clean 'url_id'
df['url_id'] = df['url_id'].str.replace('/cs/v2/estates/', '')

# Prepare 'Podlaží'
df['Podlaží'] = df['Podlaží'].str.extract('(\d+)', expand=False)
df['Podlaží'] = pd.to_numeric(df['Podlaží'], errors='coerce')

# Convert 'Celková cena' to numerical by removing whitespace and non-digit characters
df['Celková cena'] = df['Celková cena'].replace('\D', '', regex=True).astype(float)

# Correctly prepare 'Energetická náročnost budovy' by extracting the letter after "Třída"
df['Energetická náročnost budovy'] = df['Energetická náročnost budovy'].str.extract('Třída (\w)', expand=False)

# One-hot encode categorical columns, treating NaNs as a separate category
for col in one_hot_encode_cols:
    df = pd.concat([df, pd.get_dummies(df[col], prefix=col, dummy_na=True)], axis=1)
    df = df.drop(columns=[col])

# Convert binary columns to 0/1 format
for col in binary_cols:
    df[col] = df[col].notna().astype(int)

# Apply the function to the distance columns and one-hot encode them
one_hot_encoded_cols = pd.DataFrame()
for col in distance_cols:
    binned_col = bin_column(df[col])
    if binned_col is not None:
        one_hot_encoded_cols = pd.concat([one_hot_encoded_cols, pd.get_dummies(binned_col, prefix=col)], axis=1)

# Concatenate the one-hot encoded columns with the original dataframe
df = pd.concat([df, one_hot_encoded_cols], axis=1)

df.head()

,url_id,Celková cena,Podlaží,Užitná plocha,Sklep,Parkování,Výtah,Latitude,Longitude,Večerka,...,Obchod_Far,Obchod_Very Far,Lékárna_Very Close,Lékárna_Close,Lékárna_Far,Lékárna_Very Far,Metro_Very Close,Metro_Close,Metro_Far,Metro_Very Far
0,722035788,6832000.0,2.0,48.0,1,1,1,50.049548,14.462156,1043.0,...,1,0,0,0,0,1,0,0,1,0
1,1669567564,13254000.0,1.0,188.0,1,0,1,50.106655,14.482849,961.0,...,0,1,0,0,0,1,0,0,1,0
2,2418832972,6361000.0,1.0,53.0,0,1,0,50.296096,16.359198,113.0,...,0,0,0,0,0,0,0,0,0,0
3,3930260556,2214000.0,5.0,35.0,1,1,1,50.723614,15.189542,185.0,...,1,0,0,1,0,0,0,0,0,0
4,4078400588,17102000.0,3.0,82.0,0,0,1,50.073452,14.426158,467.0,...,0,0,1,0,0,0,1,0,0,0


In [14]:
df.head()


,url_id,Celková cena,Podlaží,Užitná plocha,Sklep,Parkování,Energetická náročnost budovy,Výtah,Latitude,Longitude,...,Obchod_Far,Obchod_Very Far,Lékárna_Very Close,Lékárna_Close,Lékárna_Far,Lékárna_Very Far,Metro_Very Close,Metro_Close,Metro_Far,Metro_Very Far
0,722035788,6832000.0,2.0,48.0,1,1,NaN,1,50.049548,14.462156,...,1,0,0,0,0,1,0,0,1,0
1,1669567564,13254000.0,1.0,188.0,1,0,NaN,1,50.106655,14.482849,...,0,1,0,0,0,1,0,0,1,0
2,2418832972,6361000.0,1.0,53.0,0,1,78,0,50.296096,16.359198,...,0,0,0,0,0,0,0,0,0,0
3,3930260556,2214000.0,5.0,35.0,1,1,NaN,1,50.723614,15.189542,...,1,0,0,1,0,0,0,0,0,0
4,4078400588,17102000.0,3.0,82.0,0,0,NaN,1,50.073452,14.426158,...,0,0,1,0,0,0,1,0,0,0


In [7]:
df.to_csv("test_2023-06-25_data_cleaned.csv", index=False)